In [4]:
import pandas as pd
pd.set_option('max_colwidth', 1000)
df = pd.read_json("017_after_kaldi.jsonl", lines = True)


In [5]:
df.columns

Index(['file', 'start', 'end', 'asr_transcription', 'webpage_transcript',
       'ratio', 'text', 'kaldi_transcript', 'kaldi_words', 'kaldi_word_starts',
       'kaldi_word_ends'],
      dtype='object')

In [9]:
df[df.kaldi_transcript == ""].shape[0] / df.shape[0]

0.15744269037682213

We have 15% of instances where Kaldi could not find transcriptions

In [10]:
c = df[df.kaldi_transcript != ""]
(c.end - c.start ).sum()/60**2


68.06911111111111

The remaining parts make up for 68 hours of data. Let us see how the transcriptions look like and if they are stable:

In [44]:
df[["file","asr_transcription", "kaldi_transcript"]].sample(5)


,file,asr_transcription,kaldi_transcript
2450,audio/lKMA0UZrmEs.wav,možda smo čak jedini festivali koji 90 programa ili 95 čak iz prethodne godine ne ponovi a li se dogodilo da ostanemo uskraćeni za neka velika imena zbog nedostatka o jest jeste to se konstantno dešava ibismo ona primjer gledali ide dobijate više stvar,smo jedini festival koji devedeset posto programa ili devedeset pet posto iz prethodne godine ne ponovi da li se dogodi da budemo uskraćeni za neka velika imena zbog nedostatka novca jeste to se konstantno dešava koga bismo gledali da dobijete više para
11697,audio/_BzZf0fGg0E.wav,u osopas biloriji ef gering načilog jas osao fanse ozeboižondj ju apas beloriji nealdet sipersmstsignifikannačilog jasta etskostih kameing navaj,
137,audio/2QShGnPuVjQ.wav,ovdje unišću i rezultat ti svih tih rasprava jeste jedan set tri,nišu i rezultat svih tih rasprava je jedan set tri
13077,audio/ggDpDyETtfk.wav,od 2004. godine je predsjednik opšteniška banja a predao je i listu sa svojim koalicijnim partnerima za predstojeće izbore,od dvije tisuće četiri je predsednik opštine niška banja a predao je i listu sa svojim koalicionim partnerima za predstojeće izbore
4943,audio/Vr6VEkG50qE.wav,al mi i dalje zagovaramo to kao preventivna mjera i ono što nam se dopada da ovdje u srbiji je aktualno taj marijev zakon da nateramo narodne izabranike da i to prihvate,mi i dalje zagovaramo to kao preventivna mera i ono što nam se dopada da je ovde u srbiji aktuelno taj marijin zakon da nateramo narodne izabranike da i to prihvate


Let's check if any files are missing:

In [29]:
df["segment_file"] = df.apply(lambda row:
    row["file"].replace(".wav", "").replace("audio", "seg_audio") + f'__{row["start"]:.1f}__{row["end"]:.1f}__.wav',
    axis=1
)

for file in df.segment_file:
    from pathlib import Path
    p = Path(file)
    if not p.exists():
        print("Segment", p, "does not exist!")



Optional step:

delete rows where we have empty kaldi outputs and delete corresponding audio files:

In [42]:
filtered_df = df[df.kaldi_transcript != ""]

loc = Path("seg_audio")
for segmentfile in loc.glob("*.wav"):
    if str(segmentfile) not in filtered_df["segment_file"]:
        # segmentfile.unlink()
        continue

Let's transform the string representations into something more pythonic:

In [39]:
from ast import literal_eval

filtered_df.loc[:, "kaldi_words"] = filtered_df.kaldi_words.apply(literal_eval)
filtered_df.loc[:, "kaldi_word_ends"] = filtered_df.kaldi_word_ends.apply(literal_eval)
filtered_df.loc[:, "kaldi_word_starts"] = filtered_df.kaldi_word_starts.apply(literal_eval)


/home/rupnik/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [41]:
filtered_df.to_json("018_juzne_vesti.jsonl", lines=True, orient="records")